# Análise de Músicas do Spotify

In [31]:
import pandas as pd
import requests
import spotipy
import re
import copy

In [2]:
credentials = spotipy.oauth2.SpotifyClientCredentials()
spotify = spotipy.Spotify(client_credentials_manager=credentials)

In [58]:
def chunks(lst, n):
    """Yield successive n-sized chunks from list.
    Props to stackoverflow: https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """
    
    for i in range(0, len(lst), n):
        
        yield lst[i:i + n]

def get_uri_from_link(link: str) -> str:
    """Extrai o URI de uma playlist do link dela"""

    uri = re.search(r"\/.{22}\?", link)

    try:
        uri = uri.group()[1:-1]
    
    except AttributeError:
        print(f"Não há URI para o link passado: {link}")

    return uri

def get_artists_from_playlist(playlist_uri: str) -> dict:
    """Extrai os artistas de uma playlist e os guarda em um dicionario para uso posterior
    
    :params:
        playlist_uri: Playlist que sera analisada
    
    :returns:
        artists: Dict("URI": ["URI Artista"], "NOME": ["Nome do Artista"])
    """

    artists = dict(uri_artista=[], nome_artista=[])
    playlist_tracks = spotify.playlist_tracks(playlist_id=playlist_uri)

    for track_data in playlist_tracks['items']:

        if track_data['track']:

            artists["uri_artista"] = artists["uri_artista"] + [track_data['track']['artists'][0]['uri']]
            artists["nome_artista"] = artists["nome_artista"] + [track_data['track']['artists'][0]['name']]

    return artists

def get_tracks_from_playlist(playlist_uri: str) -> dict:
    """Extrai as musicas de uma playlist e os guarda em um dicionario para uso posterior
    
    :params:
        playlist_uri: Playlist que sera analisada
    
    :returns:
        musics: Dict("URI_ARTISTA": ["URI Artista"], "URI_MUSICA": ["URI da Musica"], "NOME_MUSICA": ["Nome da Musica"], "POPULARIDADE": ["Popularidade da Musica (X - X)"], "DURACAO": ["Duracao da Musica (s)"])
    """

    musics = dict(uri_artista=[], uri_musica=[], nome_musica=[], popularidade=[], duracao_musica=[])
    playlist_tracks = spotify.playlist_tracks(playlist_id=playlist_uri)

    for track_data in playlist_tracks['items']:

        if track_data['track']:

            musics['uri_artista'].append(track_data['track']['artists'][0]['uri'])
            musics['uri_musica'].append(track_data['track']['uri'])
            musics['nome_musica'].append(track_data['track']['name'])
            musics['popularidade'].append(track_data['track']['popularity'])
            musics['duracao_musica'].append(track_data['track']['duration_ms'])
    
    return musics

def enrich_data_tracks(tracks: dict) -> dict:
    """Enriquece os dados que possuimos sobre as musicas, trazendo caracteristicas como dancabilidade, altura, etc.

    :params:
        tracks: Dicionario que obtemos da funcao get_tracks_from_playlist()
    
    :return:
        tracks: Dicionario com as informacoes novas
    """
    
    enriched_tracks = copy.deepcopy(tracks)

    enriched_tracks['danceability'] = []
    enriched_tracks['energy'] = []
    enriched_tracks['loudness'] = []
    enriched_tracks['acousticness'] = []
    enriched_tracks['tempo'] = []

    if enriched_tracks['uri_musica']:

        for track_uri in chunks(enriched_tracks['uri_musica'], 99):
            
            track_data = spotify.audio_features(track_uri)

            for track in track_data:
                
                enriched_tracks['danceability'].append(track['danceability'])
                enriched_tracks['energy'].append(track['energy'])
                enriched_tracks['loudness'].append(track['loudness'])
                enriched_tracks['acousticness'].append(track['acousticness'])
                enriched_tracks['tempo'].append(track['tempo'])
    
    return enriched_tracks
    

In [59]:
playlist_uri = get_uri_from_link("https://open.spotify.com/playlist/37i9dQZF1DXco6WE2xPt0R?si=105d1f212b914c05")

artists = get_artists_from_playlist(playlist_uri)
tracks = get_tracks_from_playlist(playlist_uri)
tracks_info = enrich_data_tracks(tracks)


In [60]:
df = pd.DataFrame(tracks_info)
df.head()

,uri_artista,uri_musica,nome_musica,popularidade,duracao_musica,danceability,energy,loudness,acousticness,tempo
0,spotify:artist:7tjVFCxJdwT4NdrTmjyjQ6,spotify:track:3xRH6FPVtheDyNla3T1FE7,Hrs and Hrs,68,204316,0.511,0.525,-4.595,0.6040,139.885
1,spotify:artist:5Vuvs6Py2JRU7WiFDVsI7J,spotify:track:1DOgkaR5Gqa8JtPEIbQORC,Over,77,205275,0.677,0.541,-6.057,0.4800,103.369
2,spotify:artist:01XujQ0Nax028Ufr2jl7vd,spotify:track:7aiVxe226qNgGf16rsv9Ip,ILM,27,236493,0.811,0.591,-7.317,0.0924,140.057
3,spotify:artist:1XkoF8ryArs86LZvFOkbyr,spotify:track:3xZ2C6ccqjPQ5f3gQzqPM2,Good Morning Gorgeous (feat. H.E.R.),60,174636,0.639,0.559,-6.749,0.2780,131.734
4,spotify:artist:3QTDHixorJelOLxoxcjqGx,spotify:track:0hZIaQMGqu41dozEwqVa1b,Satisfaction,56,130981,0.685,0.577,-4.655,0.6600,110.326
